# Lab 3: Clustering

Clustering is a very common _unsupervised_ machine learning technique. This means that during clustering we do not have class label information, but instead try to find observations that share similar characteristics.

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_wine
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
X, y = load_wine(return_X_y=True, as_frame=True)
X_standardized = StandardScaler().fit_transform(X)

We will use a wine dataset that contains the results of a chemical analysis of wines grown in a specific area of Italy.

Let's explore what the data looks like:

In [ ]:
X

In [ ]:
n_col = 3
n_row = math.ceil(len(X.columns) / n_col)

fig, axes = plt.subplots(n_row, n_col, figsize=(10, 10))

for col, ax in zip(X.columns, axes.ravel()):
    sns.histplot(data=X, x=col, ax=ax)
    sns.despine(ax=ax)
    
plt.tight_layout()

plt.show()
plt.close()

Looks good! There are 13 different features. We can also use PCA to reduce the data to two dimensions for plotting.

In [ ]:
pca = PCA(n_components=2).fit(X_standardized)

In [ ]:
fig, ax = plt.subplots()

X_reduced = pca.transform(X_standardized)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1])
ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")

sns.despine(ax=ax)

plt.show()
plt.close()

## k-Means

**Now let's do some clustering.** We will use k-means clustering with $k = 5$ initially.

In [ ]:
kmeans = KMeans(n_clusters=5, n_init="auto", random_state=0)

Let's show the cluster assignments on the PCA plot.

In [ ]:
kmeans.fit_predict(X)

fig, ax = plt.subplots()

X_reduced = pca.transform(X_standardized)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=kmeans.labels_)

ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")

sns.despine(ax=ax)

plt.show()
plt.close()

This looks very weird! The cluster assignments don't correspond to our intuition.
<br><br>
<details>
<summary><b>What went wrong?</b> (Click to expand)</summary>
    
**We didn't standardize our data prior to k-Means clustering!** Because our features are on different scales, features with bigger magnitudes dominate the Euclidean distance calculations and k-Means won't give the expected result.

_Note:_ We sneakily did standardize our features prior to the PCA transformation as we have seen during the previous lecture. If PCA would have been performed on non-standardized data as well (which is incorrect), the clusters on the plot would look more natural.

Let's try this again, but now standardize our features prior to k-Means clustering!
</details>

In [ ]:
kmeans.fit(X_standardized)

fig, ax = plt.subplots()

X_reduced = pca.transform(X_standardized)
centroids = pca.transform(kmeans.cluster_centers_)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=kmeans.labels_)
ax.scatter(centroids[:, 0], centroids[:, 1], s=100, c="red", marker="X")

ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")

sns.despine(ax=ax)

plt.show()
plt.close()

This makes a lot more sense!
<br><br>
<details>
<summary>
<b>Why does there still seem to be a mismatch between the cluster centroids and some of the cluster labels?</b> (Click to expand)
</summary>

We performed k-Means clustering in the original, 13-dimensional space, whereas plotting happens in two dimensions. PCA causes some loss of information, which makes it seem as if some cluster assignments are incorrect.
</details>

We managed to successfully cluster, but is five clusters the optimal value?
<details>
<summary>
<b>How do we determine the number of clusters?</b> (Click to expand)
</summary>

We can use the **elbow method** to figure out how many clusters we need. Let's run k-Means for $k=2$ to $k=10$ and plot the cost function (sum of squared distances) for all clusterings.
</details>

In [ ]:
cost = [
    KMeans(n_clusters=k, n_init="auto", random_state=0).fit(X_reduced).inertia_
    for k in range(2, 11)
]

fig, ax = plt.subplots()

ax.plot(range(2, 11), cost, marker="o")
ax.set_xlabel("k")
ax.set_ylabel("Cost function")

sns.despine(ax=ax)

plt.show()
plt.close()

Now let's run clustering again with your optimal value of $k$ and plot the results.

In [ ]:
# Specify the number of clusters here:
n_clusters = 3# TODO

kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=0).fit(X_standardized)

fig, ax = plt.subplots()

X_reduced = pca.transform(X_standardized)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=kmeans.labels_)

ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")

sns.despine(ax=ax)

plt.show()
plt.close()

That looks like excellent clustering! We managed to find the optimal number of clusters using the elbow method. For this toy example, a good value was relatively straightforward to determine, but for more complex data a definitive answer might not always be available. Additionally, to use the elbow method, we had to run k-Means for multiple values of $k$, leading to runtime overhead.

Because we know the ground truth for this dataset, we do know class labels in this case. _This is normally not the case when using clustering!_ Let's compare our clustering results to the ground truth labeling.

We see indeed that there are three classes in our data, which k-Means was able to recover!

In [ ]:
fig, ax = plt.subplots()

X_reduced = pca.transform(X_standardized)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y)

ax.set_xlabel("PCA1")
ax.set_ylabel("PCA2")

sns.despine(ax=ax)

plt.show()
plt.close()

## Hierarchical clustering

Now we'll perform hierarchical clustering on the same data and plot the dendrogram. **Can you derive the optimal number of clusters from the dendrogram?**

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
agg_cluster = AgglomerativeClustering(
    distance_threshold=0, n_clusters=None, linkage="ward"
)

agg_cluster.fit(X_standardized)

fig, ax = plt.subplots(figsize=(10, 5))

plot_dendrogram(agg_cluster, truncate_mode="level", p=100)

ax.set_xticklabels([])
ax.set_xlabel("samples")
ax.set_ylabel("Cluster distance")

sns.despine(ax=ax)

plt.show()
plt.close()

Scikit-learn uses **Ward linkage** by default (not discussed during class), which minimizes the variance of clusters being merged.

**Try different values for the linkage method and discuss how this changes the dendrogram.** Try the following linkage methods:

- average
- complete
- single